<a href="https://colab.research.google.com/github/SnehalYu/Query-Auto-Completion-/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Training the model

import numpy as np
# Specify the file paths
file_path1 = "/content/qac_background.txt"
file_path2 = "/content/qac_training.tsv"
file_path3 = "/content/qac_training.tsv"
file_path4 = "/content/qac_validation.tsv"
# Read the files
with open(file_path1, 'r') as file:
    content1 = file.read()

with open(file_path2, 'r') as file:
    content2 = file.read()

with open(file_path3, 'r') as file:
    content3 = file.read()

with open(file_path4, 'r') as file:
    content4 = file.read()

# Combine the content
trainingset = content1 + content2
trainset = trainingset.lower()  # Convert to lowercase
dataset=trainset+content3.lower()+content4.lower()
# Limit the trainingset size
#trainset = trainset[:50000]

trainset=list(trainset)
trainset.insert(0,'<s>')
trainset.insert(0,'<s>')
trainset.append('</s>')
trainset.append('</s>')

# Create a list of characters
characters = sorted(set(dataset))
characters.append('<s>')
characters.append('</s>')
char_index = {char: index for index, char in enumerate(characters)}

#Count bigrams and trigrams
dict_two = {}
dict_three = {}
trainsetcount = len(trainset)

for i in range(trainsetcount - 1): # -1 modified to -3
    if trainset[i]=='\n':
        pair_two = tuple(['\n','</s>'])
        pair_three = tuple(['\n', '</s>', '</s>'])
    elif trainset[i+1]=='\n':
        pair_two = tuple([trainset[i],'\n'])
        pair_three = tuple([trainset[i],'\n', '</s>'])
    elif i>0 and trainset[i-1] =='\n':
        pair_two = tuple(['<s>','<s>'])
        pair_three = tuple(['<s>','<s>',trainset[i] ])
    elif i>1 and trainset[i-2] =='\n':
        pair_two = tuple(['<s>', trainset[i-1]])
        pair_three = tuple(['<s>',trainset[i-1], trainset[i] ])
    else:
        pair_two = tuple(trainset[i:i + 2])
        pair_three = tuple(trainset[i:i + 3])

    ## pair_two, pair_three declared above
    if pair_two in dict_two:
        dict_two[pair_two] += 1
    else:
        dict_two[pair_two] = 1
    if pair_three in dict_three:
        dict_three[pair_three] += 1
    else:
        dict_three[pair_three] = 1
dict_bi_to_num={value: index for index, value in enumerate(dict_two)}
dict_num_to_bi={index: value for index, value in enumerate(dict_two)}

# Create arrays for bigram and trigram probabilities
num_bigrams = len(dict_two)
num_chars = len(characters)

table = np.zeros((num_bigrams, num_chars))

for i, bigram in enumerate(dict_two):
    for j, char in enumerate(characters):
        trigram = (bigram) + (char,)
        if trigram in dict_three:
            num = dict_three[trigram] + 1
        else:
            num = 1
        denom = dict_two[bigram] + num_chars
        table[i, j] = num / denom # row:bigram, col:trigram

